In [1]:
from modeler.modeler import Modeler as m
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime
from tqdm import tqdm

In [2]:
market = Market()
strategy = Strategy()

In [3]:
market.connect()
prices = market.retrieve("prices")
financials = market.retrieve("unified_financials")
sp5 = market.retrieve("sp500")
market.disconnect()
strategy.connect()
categories = strategy.retrieve("predicted_stock_categories")
strategy.disconnect()

In [4]:
categories["prediction"].unique()

array([None, '100Materials', '100Information Technology',
       '100Consumer Discretionary', '500Communication Services',
       '200Industrials', '100Consumer Staples', '500Industrials',
       '200Consumer Discretionary', '100Financials', '100Health Care',
       '200Information Technology', '500Health Care', '100Industrials',
       '200Health Care', '500Information Technology', '100Real Estate',
       '100Energy', '500Consumer Discretionary',
       '100Communication Services', '100Utilities', '200Real Estate',
       '200Materials', '200Financials', '200Consumer Staples',
       '200Communication Services', '500Real Estate', '200Energy',
       '500Financials', '200Utilities', '500Consumer Staples',
       '500Materials'], dtype=object)

In [7]:
categories["prediction"].unique()

array(['None', '100Materials', '100Information Technology',
       '100Consumer Discretionary', '500Communication Services',
       '200Industrials', '100Consumer Staples', '500Industrials',
       '200Consumer Discretionary', '100Financials', '100Health Care',
       '200Information Technology', '500Health Care', '100Industrials',
       '200Health Care', '500Information Technology', '100Real Estate',
       '100Energy', '500Consumer Discretionary',
       '100Communication Services', '100Utilities', '200Real Estate',
       '200Materials', '200Financials', '200Consumer Staples',
       '200Communication Services', '500Real Estate', '200Energy',
       '500Financials', '200Utilities', '500Consumer Staples',
       '500Materials'], dtype=object)

In [8]:
prices["date"] = pd.to_datetime(prices["date"])
prices["date"]
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [9]:
financials["year"] = [row[1]["year"] + 1 if row[1]["quarter"] == 4 else row[1]["year"] for row in financials.iterrows()]
financials["quarter"] = [1 if row[1]["quarter"] == 4 else row[1]["quarter"] + 1 for row in financials.iterrows()]

In [10]:
financials = financials.groupby(["year","quarter","ticker"]).mean().reset_index()

In [11]:
valid = []
for ticker in financials["ticker"].unique():
    if financials[financials["ticker"]==ticker].index.size == 49:
        valid.append(ticker)

In [21]:
financials.columns

Index(['year', 'quarter', 'ticker',
       'AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'Assets',
       'AssetsCurrent', 'CashAndCashEquivalentsAtCarryingValue',
       'EarningsPerShareBasic', 'EarningsPerShareDiluted',
       'EntityCommonStockSharesOutstanding', 'IncomeTaxExpenseBenefit',
       'LiabilitiesAndStockholdersEquity', 'LiabilitiesCurrent',
       'NetIncomeLoss', 'OtherAssetsNoncurrent',
       'RetainedEarningsAccumulatedDeficit', 'StockholdersEquity',
       'CommonStockValue', 'Goodwill', 'PropertyPlantAndEquipmentNet'],
      dtype='object')

In [17]:
quarterly_sets = []
start = 2012
end = 2022
prices = prices[(prices["year"] >= start) & (prices["year"] <= end)]
for ticker in tqdm(sp5["Symbol"].unique()):
    ticker_data = prices[prices["ticker"]==ticker]
    quarterly = ticker_data.groupby(["year","quarter"]).mean().reset_index()
    quarterly["y"] = quarterly["adjClose"].shift(-1)
    quarterly.dropna(inplace=True)
    quarterly["ticker"] = ticker
    quarterly_sets.append(quarterly)
initial_data = pd.concat(quarterly_sets)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:04<00:00,  7.86it/s]


In [22]:
financials["date"] = [datetime(row[1]["year"], row[1]["quarter"] * 3 -  2, 1) for row in financials.iterrows()]
initial_data["date"] = [datetime(row[1]["year"], row[1]["quarter"] * 3 -  2, 1) for row in initial_data.iterrows()]

In [19]:
initial_data[(initial_data["year"]==2021) & (initial_data["quarter"]==4)]

,year,quarter,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor,week,y,ticker,date


In [100]:
financial_factors = ['AccumulatedOtherComprehensiveIncomeLossNetOfTax', 'Assets',
       'AssetsCurrent', 'CashAndCashEquivalentsAtCarryingValue',
       'EarningsPerShareBasic', 'EarningsPerShareDiluted',
       'EntityCommonStockSharesOutstanding', 'IncomeTaxExpenseBenefit',
       'LiabilitiesAndStockholdersEquity', 'LiabilitiesCurrent',
       'NetIncomeLoss', 'OtherAssetsNoncurrent',
       'RetainedEarningsAccumulatedDeficit', 'StockholdersEquity','CommonStockValue', 'Goodwill',
       'PropertyPlantAndEquipmentNet']
yearly_gap = 1
analysis = []
for category_training_year in range(0,5):
    subset_categories = categories[categories["training_years"]==category_training_year]
    labels = initial_data.merge(subset_categories,on=["year","quarter","ticker"],how="left")
    factors = financials.merge(subset_categories,on=["year","quarter","ticker"],how="left")
    for model_training_year in range(1,6):
        for year in range(2018,2022):
            for quarter in range(1,5):
                quarterly_categories = labels[(labels["year"]==year) & (labels["quarter"]==quarter)]
                for category in labels["prediction"].unique():
                    try:
                        if category == "None":
                            continue
                        else:
                            category_tickers = labels[labels["prediction"]==category]["ticker"].unique()
                            training_factors = factors[factors["ticker"].isin(list(category_tickers))]
                            training_factors.sort_values("date",inplace=True)
                            if quarter == 1:
                                new_quarter = 4
                                new_year = year - 1
                            else:
                                new_quarter = quarter - 1
                                new_year = year
#                             first_index = training_factors[(training_factors["year"] == year - model_training_year-1) & (training_factors["quarter"]==quarter)].index.values.tolist()[0]
                            last_index = training_factors[(training_factors["year"] == new_year) & (training_factors["quarter"]==new_quarter)].index.values.tolist()[0]
                            spliced_training = training_factors.iloc[:last_index]
                            relevant_factors = spliced_training[(spliced_training["year"] < year) & (spliced_training["year"]>=year-model_training_year)]
                            training_labels = labels[labels["ticker"].isin(list(category_tickers))]
                            training_labels.sort_values("date",inplace=True)
#                             first_index = training_labels[(training_labels["year"] == year - model_training_year-1) & (training_labels["quarter"]==quarter)].index.values.tolist()[0]
                            last_index = training_labels[(training_labels["year"] == new_year) & (training_labels["quarter"]==new_quarter)].index.values.tolist()[0]
                            training_labels = training_labels.iloc[:last_index]
                            relevant_labels = training_labels[(training_labels["year"] < year) & (training_labels["year"]>=year-model_training_year)]
                            prediction_data = training_factors[(training_factors["year"]==year) & (training_factors["quarter"]==quarter)]
                            relevant = relevant_factors.merge(relevant_labels,on=["year","quarter","ticker"],how="left").dropna()
                            X = relevant[financial_factors]
                            y = relevant["adjClose"]
                            xgb_models = m.xgb_regression({"X":X,"y":y})
                            sk_models = m.sk_regression({"X":X,"y":y})
                            sk_models.append(xgb_models)
                            models = pd.DataFrame(sk_models)
                            model = models.sort_values("score",ascending=False).iloc[0]
                            sim = prediction_data
                            sim["quarterly_price_regression_prediction"] = model["model"].predict(sim[financial_factors])
                            sim["score"] = model["score"].item()
                            sim = sim[["year","quarter","ticker","quarterly_price_regression_prediction","score"]]
                            sim["model_training_year"] = model_training_year
                            sim["category_training_year"] = category_training_year
                            if sim.index.size > 1:
                                strategy.connect()
                                strategy.store("quarterly_short_sim",sim)
                                strategy.disconnect()
                    except Exception as e:
                        print(year,quarter,category,str(e)) 
                        continue

With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 1 100Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 3 100Information Technology With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 4 200Industrials With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Ad

With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2020 4 100Communication Services With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2021 3 500Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2021 4 100Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Ad

With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 1 200Real Estate With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 1 200Consumer Discretionary With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 1 200Consumer Staples With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empt

With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 2 200Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 2 500Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 2 100Real Estate With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of

With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 3 500Information Technology With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 3 100Materials With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 3 100Utilities With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust

...

2018 1 500Communication Services With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 1 200Information Technology With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 1 500Financials With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 1 100Financials With n_samples=0, test_size=0.25 and train_size=0.75, the result

With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 2 100Materials With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 2 200Consumer Staples With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 2 500Industrials With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any

2018 3 100Communication Services With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 3 200Communication Services With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 3 500Materials With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2018 3 200Real Estate With n_samples=0, test_size=0.25 and train_size=0.75, the result

With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2019 1 100Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2019 1 200Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2019 1 100Information Technology With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Ad

2020 4 nan list index out of range
With n_samples=1, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2020 4 100Industrials With n_samples=1, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2020 4 500Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2021 1 nan list index out of range
With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
2021 1 100Health Care With n_samples=0, test_size=0.25 and train_size=0.75, the resulting train set will be empty. Adjust any of the aforementioned parameters.
With n_samples=0, test_size=0.25 and